In [18]:
%load_ext autoreload
%autoreload 2

from tweedejaars_project import load_df
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *


df = load_df()

print(df)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
                        datetime  import_capacity  upward_dispatch_published  \
0      2023-12-29 00:00:00+00:00             65.4                        0.0   
1      2023-12-29 00:01:00+00:00             65.4                        0.0   
2      2023-12-29 00:02:00+00:00             65.4                        0.0   
3      2023-12-29 00:03:00+00:00             65.4                        0.0   
4      2023-12-29 00:04:00+00:00             65.4                        0.0   
...                          ...              ...                        ...   
220241 2024-05-29 22:41:00+00:00           1982.6                      167.0   
220242 2024-05-29 22:42:00+00:00           1982.6                      165.0   
220243 2024-05-29 22:43:00+00:00           1982.6                      162.0   
220244 2024-05-29 22:44:00+00:00           1982.6                      158.0   
220245 2024-05-29 22:45:00+00:00

In [19]:
features = [
    "import_capacity",
    "settlement_price_bestguess",
]

train_x = df[features][:165195]
train_y = df['target_two_sided_ptu'][:165195]

test_x = df[features][165195:]
test_y = df['target_two_sided_ptu'][165195:]

# Initialize the Decision Tree Classifier
clf = HistGradientBoostingClassifier()

# Train the classifier
clf.fit(train_x, train_y)

y_pred = clf.predict(test_x)

# Evaluate the model
accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

print("Classification Report:")
print(classification_report(test_y, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_y, y_pred))

0.9785290003814645
Classification Report:
              precision    recall  f1-score   support

       False       0.98      1.00      0.99     54028
        True       0.16      0.04      0.06      1023

    accuracy                           0.98     55051
   macro avg       0.57      0.52      0.53     55051
weighted avg       0.97      0.98      0.97     55051

Confusion Matrix:
[[53830   198]
 [  984    39]]


In [20]:
def compute_penalty(df: pd.DataFrame, true, pred, ids):
    energy = 100 / 60
    df['pred'] = pred
    df['id'] = ids
    df['true'] = true
    df['combined'] = False
    df.loc[~df['pred'], 'combined'] = df.loc[~df['pred'], 'naive_strategy_action']
    agg_dict = {
        'min_price_published': 'min',
        'max_price_published': 'max',
        'combined': 'sum',
        'true': 'any'
    }
    flat_df = df.groupby('id').aggregate(agg_dict)
    view = flat_df[flat_df['true']]
    penalty = energy * view['max_price_published'] * view['combined']
    return penalty.sum()


# def compute_penalty(df, pred, ids):
#     total_penalty = 0

#     pred_df = pd.DataFrame({
#         "ptu_id": ids,
#         "target_two_sided_ptu_pred": pred
#     })
    
#     df = df.set_index("ptu_id")
#     pred_df = pred_df.set_index("ptu_id")
    
#     df["target_two_sided_ptu_pred"] = pred_df["target_two_sided_ptu_pred"]

#     df = df.reset_index()

#     grouped = df.groupby("ptu_id")

#     for _, job_df in grouped:
#         if job_df['target_two_sided_ptu'].any():
#             count = 0
#             positive_found = False

#             job_df = job_df.reset_index(drop=True)

#             for _, row in job_df.iloc[2:].iterrows():
#                 min_price = row["min_price_published"]
#                 max_price = row["max_price_published"]

#                 if min_price < 0 and not positive_found:
#                     count += 1

#                 if row['target_two_sided_ptu']:
#                     if min_price < 0 and not positive_found:
#                         count -= 1

#                 if pd.notna(max_price) and max_price > 0:
#                     positive_found = True

#             max_price = job_df["max_price_published"].max()
#             if pd.notna(max_price):
#                 penalty = ((100 / 60) * max_price).round(-1) * count
#                 total_penalty += penalty

#     return total_penalty

# def compute_reward(df, pred, ids):
#     total_reward = 0

#     pred_df = pd.DataFrame({
#         "ptu_id": ids,
#         "target_two_sided_ptu_pred": pred
#     })
    
#     df = df.set_index("ptu_id")
#     pred_df = pred_df.set_index("ptu_id")
    
#     df["target_two_sided_ptu_pred"] = pred_df["target_two_sided_ptu_pred"]

#     df = df.reset_index()

#     grouped = df.groupby("ptu_id")

#     for _, job_df in grouped:
#         if not job_df['target_two_sided_ptu'].any():

#             job_df = job_df.reset_index(drop=True)

#             if pd.notna(max_price):
#                 reward = ((100 / 60) * min_price).round(-1) * count
#                 total_reward += reward

#     return total_reward

compute_penalty(df, df['target_two_sided_ptu'], df['target_two_sided_ptu'], df['ptu_id'])

0.0

In [23]:
data = {
    'Time': [
        '11:30', '11:31', '11:32', '11:33', '11:34', '11:35', '11:36', 
        '11:37', '11:38', '11:39', '11:40', '11:41', '11:42', '11:43', 
        '11:44', '11:45'
    ],
    'min_price_published': [
        -100, np.NaN, 0, -100, -150, 
        -200, -300, -100, np.NaN, np.NaN, 
        np.NaN, np.NaN, np.NaN, -30, -100, 0
    ],
    'max_price_published': [
        np.NaN, 20, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, 
        40, 50, 90, 200, 80, 
        np.NaN, np.NaN, np.NaN
    ],
    'target_two_sided_ptu': [
        True, False, False, True, True, False, False, True, False, False,
        False, False, False, False, False, False
    ],
    'pred': [
        True, False, False, True, True, False, False, True, False, False,
        False, False, False, False, False, False
    ],
    'naive_strategy_action': [
        False, False, False, True, True, True, True, True, False, False,
        False, False, False, False, False, False,
    ],
    'ptu_id':
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
}

df = pd.DataFrame(data)


compute_penalty(df, df['target_two_sided_ptu'], df['pred'], df['ptu_id'])

666.6666666666667